In [1]:
from lightning import LitCNN
from utils.transforms import get_transform
from datasets import BeamDataset, collate_fn

from torch.utils import data as torch_data
from pytorch_lightning import Trainer
import math
import copy

/opt/conda/envs/eagles/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# ssh -N -f -L localhost:16006:localhost:6006 kamranzolfonoon@35.203.109.50
%load_ext tensorboard
%tensorboard --logdir lightning_logs/ --port 6006

In [3]:
dataset = BeamDataset("/home/kamranzolfonoon/dev/eagle-images/beam_large_batch", "/home/kamranzolfonoon/dev/eagle-images/beam_large_batch/annotations/annotations.json", get_transform(train=True))

# TODO: Remove jitter transform from test dataset
train_dataset = torch_data.Subset(dataset, range(0, math.floor(len(dataset) * 0.8)))
test_dataset = torch_data.Subset(dataset, range(math.floor(len(dataset) * 0.8), len(dataset)))
test_dataset.dataset = copy.deepcopy(test_dataset.dataset)
test_dataset.dataset.transforms = get_transform(train=False)

train_loader = torch_data.DataLoader(train_dataset, batch_size=4, shuffle=True, num_workers=1, collate_fn=collate_fn)
test_loader = torch_data.DataLoader(test_dataset, batch_size=4, shuffle=False, num_workers=1, collate_fn=collate_fn)

In [4]:
# num_classes = 4

# model_name = "fasterrcnn_resnet50_fpn_v2"

# model_params = {
#     "num_classes": num_classes,
#     "pretrained": True,
#     "frozen": False
# }

# optimizer_name = "SGD"

# optimizer_params = {
#     "lr": 0.005,
#     "momentum": 0.9,
#     "weight_decay": 0.0005
# }

# scheduler_name = "StepLR"

# scheduler_params = {
#     "step_size": 10,
#     "gamma": 0.1
# }

num_classes = 4

model_name = "fasterrcnn_resnet50_fpn_v2"

model_params = {
    "num_classes": num_classes,
    "pretrained": True,
    "frozen": False
}

optimizer_name = "SGD"

optimizer_params = {
    "lr": 0.005,
    "momentum": 0.9,
    "weight_decay": 0.0005
}

scheduler_name = "MultiStepLR"

scheduler_params = {
    "milestones": [10, 20],
    "gamma": 0.1
}

model = LitCNN(model_name, model_params, optimizer_name, optimizer_params, scheduler_name, scheduler_params)

In [5]:
trainer = Trainer(accelerator='gpu', devices=1, max_epochs=30)
trainer.fit(model, train_loader, test_loader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name           | Type                 | Params
--------------------------------------------------------
0 | model          | FasterRCNN           | 41.3 M
1 | validation_map | MeanAveragePrecision | 0     
--------------------------------------------------------
41.1 M    Trainable params
222 K     Non-trainable params
41.3 M    Total params
165.258   Total estimated model params size (MB)


Epoch 0:   1%|          | 1/148 [00:01<03:20,  1.36s/it]                   

/opt/conda/envs/eagles/lib/python3.9/site-packages/pytorch_lightning/utilities/data.py:85: UserWarning: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 3. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
  warning_cache.warn(


Epoch 5:  78%|███████▊  | 116/148 [02:21<00:38,  1.22s/it, loss=0.233, v_num=14]

In [ ]:
import torch
from torchvision.utils import draw_bounding_boxes
from torchvision.transforms.functional import to_pil_image
from utils.visualization import visualize_image

# If checkpoint is saved, load it:
# model = LitCNN.load_from_checkpoint('/home/kamranzolfonoon/dev/eaglescnn/lightning_logs/version_0/checkpoints/epoch=29-step=3540.ckpt')
# good example: 12, 59
# tricky: 5, 80, 89, 31, 57
img, target = test_dataset[59]
visualize_image(model,img,target, "prediction.png", "labels.png")




/tmp/ipykernel_18523/1111890060.py:28: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  prediction = draw_bounding_boxes(torch.tensor(img*255,dtype=torch.uint8), boxes=boxes,
/tmp/ipykernel_18523/1111890060.py:43: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  ground_truth = draw_bounding_boxes(torch.tensor(img*255,dtype=torch.uint8), boxes=boxes,
